In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras import Model
import tensorflow as tf
from tensorflow.keras.applications.xception import Xception

import h5py

xception_weights = '../../DeepSlice/metadata/weights/xception_weights_tf_dim_ordering_tf_kernels.h5'

In [ ]:
allen_weight_path = '../../DeepSlice/metadata/weights/Allen_Mixed_Best.h5'
new_allen_weight_path = '../../DeepSlice/metadata/weights/Allen_Mixed_Best_2.h5'
temp_allen_weight_path = '../../DeepSlice/metadata/weights/Allen_Mixed_Best_temp.h5'

with h5py.File(temp_allen_weight_path,'w') as f_dest:
    with h5py.File(allen_weight_path,'r') as f_src:

        layer_names = []

        for x in f_src['xception'].items():
            print(x[0])
            name = x[0]
            #Batch normalization layers appear to have flipped beta and gamma

            f_src.copy(f_src['xception'][name],f_dest,name)

            weight_names = []
            for y in f_src['xception'][name].items():
                weight_name = y[0]
                weight_names.append(weight_name)

            #Identify batchnorm layer becuase beta and gamma are flipped for some reason
            if 'gamma:0' in weight_names:

                gamma_index = weight_names.index('gamma:0')
                beta_index = weight_names.index('beta:0')

                gamma = f_src['xception'][name]['gamma:0'][...]
                beta = f_src['xception'][name]['beta:0'][...]

                f_dest[name]['gamma:0'][...] = beta
                f_dest[name]['beta:0'][...] = gamma

                #weight_names[gamma_index] = 'beta:0'
                #weight_names[beta_index] = 'gamma:0'
            

            f_dest[name].attrs['weight_names']=weight_names

            layer_names.append(name)

        for x in f_src['xception'].attrs.keys():
            #create attribute in destination file
            f_dest.attrs['layer_names'] = layer_names


base_model = Xception(include_top=True, weights=xception_weights)

base_model = Model(inputs=base_model.input, outputs=base_model.layers[-1].output,name="xception")

base_model.load_weights(temp_allen_weight_path,skip_mismatch=True,by_name=True)

model = Sequential()
model.add(base_model)
model.add(Dense(256, activation="relu"))
model.add(Dense(256, activation="relu"))
model.add(Dense(9, activation="linear"))

model.load_weights(allen_weight_path,by_name=True,skip_mismatch=True)

model.save_weights(new_allen_weight_path)

In [ ]:
synthetic_weight_path = '../../DeepSlice/metadata/weights/Synthetic_data_final.hdf5'
new_synthetic_weight_path = '../../DeepSlice/metadata/weights/Synthetic_data_final_2.hdf5'
temp_synthetic_weight_path = '../../DeepSlice/metadata/weights/Synthetic_data_final_temp.hdf5'

with h5py.File(temp_synthetic_weight_path,'w') as f_dest:
    with h5py.File(synthetic_weight_path,'r') as f_src:

        layer_names = []

        for x in f_src['xception'].items():
            name = x[0]
            f_src.copy(f_src['xception'][name],f_dest,name)

            weight_names = []
            for x in f_src['xception'][name].items():
                weight_name = x[0]
                weight_names.append(weight_name)

            #Identify batchnorm layer becuase beta and gamma are flipped for some reason
            if 'gamma:0' in weight_names:
                gamma_index = weight_names.index('gamma:0')
                beta_index = weight_names.index('beta:0')

                gamma = f_src['xception'][name]['gamma:0'][...]
                beta = f_src['xception'][name]['beta:0'][...]

                f_dest[name]['gamma:0'][...] = beta
                f_dest[name]['beta:0'][...] = gamma

                #weight_names[gamma_index] = 'beta:0'
                #weight_names[beta_index] = 'gamma:0'
            
            f_dest[name].attrs['weight_names']=weight_names

            layer_names.append(name)

        for x in f_src['xception'].attrs.keys():
            #create attribute in destination file
            f_dest.attrs['layer_names'] = layer_names

base_model = Xception(include_top=True, weights=xception_weights)

base_model = Model(inputs=base_model.input, outputs=base_model.output,name="xception")

model = Sequential()
model.add(base_model)
model.add(Dense(256, activation="relu"))
model.add(Dense(256, activation="relu"))
model.add(Dense(9, activation="linear"))

base_model.load_weights(temp_synthetic_weight_path,by_name=True)
model.load_weights(synthetic_weight_path,by_name=True,skip_mismatch=True)

model.save_weights(new_synthetic_weight_path)

In [ ]:
#This will not produce a weight file from xception because it doesn't appropriately save teh 
#weight list


synthetic_weight_path = '../../DeepSlice/metadata/weights/Synthetic_data_final.hdf5'
new_synthetic_weight_path = '../../DeepSlice/metadata/weights/Synthetic_data_final_2.hdf5'

with h5py.File(new_synthetic_weight_path,'w') as f_dest:
    with h5py.File(synthetic_weight_path,'r') as f_src:
        for x in f_src['xception'].items():
            name = x[0]
            f_src.copy(f_src['xception'][name],f_dest,name)

base_model = Xception(include_top=True, weights=xception_weights)

base_model = Model(inputs=base_model.input, outputs=base_model.layers[-1].output,name="xception")

model = Sequential()
model.add(base_model)
model.add(Dense(256, activation="relu"))
model.add(Dense(256, activation="relu"))
model.add(Dense(9, activation="linear"))

base_model.load_weights(new_synthetic_weight_path,by_name=True)

model.load_weights(synthetic_weight_path,by_name=True,skip_mismatch=True)

model.save_weights(new_synthetic_weight_path)